A notebook for training a music generation transformer

In [1]:
import sys
sys.path.insert(0, '../src/libs')

In [2]:
# our file imports
from music_transformer import *
from transformer_training_helpers import *
from song_classification import *
from dataset import *
from torch.autograd import Variable

# musicautobot
from musicautobot.numpy_encode import *
from musicautobot.config import *
from musicautobot.music_transformer import *
from musicautobot.utils.midifile import *
from musicautobot.utils.file_processing import process_all

In [14]:
# FIXME: DONT NEED
#midi_path = '../src/mid_data_collections/mid_0_to_10000'
# numpy_path = '../numpy_path'

In [15]:
# FIXME: DONT NEED
# numpy_files = get_files(numpy_path, extensions='.npy', recurse=True); len(numpy_files)
#midi_files = get_files(midi_path, '.mid', recurse=True)
#print(midi_files[0])
#print(len(midi_files))

1177

NEW DATASET CODE

In [3]:
myDs = MidiDataset('../numpy_path', 150, 'N/A', '.npy')
dataloader = DataLoader(myDs, 1, shuffle=True)

for _, item in enumerate(dataloader):
    print(item)
    break

tensor([[[  0.,   1.,  79.,  ..., 139.,  63., 139.],
         [  8., 138.,  79.,  ..., 139.,  68., 139.],
         [ 51., 140.,   8.,  ..., 139.,  63., 139.],
         ...,
         [ 75., 138.,  70.,  ..., 138.,   8., 139.],
         [ 80., 138.,  77.,  ..., 138.,  51., 140.],
         [  8., 138.,  84.,  ..., 138.,  86., 138.]]])


END NEW DATASET CODE

In [16]:
# FIXME: DONT NEED
# data_path = './saved_data/'
# data_save_name = 'music_item_data_0-10000.pkl'
# def create_databunch(files, path):
#     #save_file.parent.mkdir(exist_ok=True, parents=True)
#     vocab = MusicVocab.create()
#     processors = [OpenNPFileProcessor(), MusicItemProcessor()]

#     #data = MusicDataBunch.from_files(midi_files, path, processors=processors, encode_position=True)
#     data = MusicDataBunch.from_files(files, path, processors=processors, encode_position=True)
#     return data

In [17]:
# FIXME: DONT NEED
# all_data = create_databunch(numpy_files, data_path); all_data

C:\Users\Trevi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\fastai\core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(a, dtype=dtype, **kwargs)


MusicDataBunch;

Train: LabelList (1060 items)
x: MusicItemList

MusicItem - (9818,)
xxbos xxpad n54 d2 n50 d2 n38 d1 xxsep d1...,
MusicItem - (6616,)
xxbos xxpad n64 d2 n61 d2 n57 d2 n52 d4...,
MusicItem - (9758,)
xxbos xxpad xxsep d7 n54 d11 xxsep d1 n64 d11...,
MusicItem - (1236,)
xxbos xxpad n76 d32 n72 d32 n67 d32 n64 d32...,
MusicItem - (7064,)
xxbos xxpad n76 d2 n72 d2 n60 d29 n57 d28...
y: LMLabelList
,,,,
Path: saved_data;

Valid: LabelList (117 items)
x: MusicItemList

MusicItem - (11442,)
xxbos xxpad xxsep d9 n55 d1 xxsep d1 n57 d1...,
MusicItem - (15824,)
xxbos xxpad xxsep d11 n89 d1 n88 d1 xxsep d1...,
MusicItem - (2552,)
xxbos xxpad n70 d10 n67 d10 n62 d10 n58 d10...,
MusicItem - (5066,)
xxbos xxpad n58 d1 n55 d1 n51 d1 n39 d4...,
MusicItem - (5798,)
xxbos xxpad n60 d128 n55 d32 n51 d32 n48 d12...
y: LMLabelList
,,,,
Path: saved_data;

Test: None

In [24]:
# the_train_data = data_gen(all_data.train_ds)
# print(the_train_data[0].src.shape)
# print(the_train_data[0].trg.shape)
# print(the_train_data[0].trg_y.shape)
# print(the_train_data[0].src_mask.shape)
# print(the_train_data[0].trg_mask.shape)

torch.Size([65, 150])
torch.Size([65, 149])
torch.Size([65, 149])
torch.Size([65, 1, 150])
torch.Size([65, 149, 149])


In [29]:
# for _, batch in enumerate(dataloader):
#     #out, w2 = model.forward(batch.src, batch.trg, 
#     #                    batch.src_mask, batch.trg_mask)
#     print(batch.shape)
#     print('beg')
#     batch = batch.long()
#     batch = batch.squeeze(0)
#     PAD = 0
#     trg = batch[:,:-1]
#     trg_y = batch[:, 1:]
#     src_mask = (batch != PAD).unsqueeze(-2)
#     trg_mask = (trg != PAD).unsqueeze(-2)
#     trg_mask = trg_mask & Variable(subsequent_mask(trg.size(-1)).type_as(trg_mask.data))
#     print(batch.shape)
#     print(trg.shape)
#     print(trg_y.shape)
#     print(src_mask.shape)
#     print(trg_mask.shape)
#     break

torch.Size([1, 36, 150])
beg
torch.Size([36, 150])
torch.Size([36, 149])
torch.Size([36, 149])
torch.Size([36, 1, 150])
torch.Size([36, 149, 149])


In [4]:
def run_epoch(data_iter, model, loss_compute):#, bad_idxs):
    "Standard Training and Logging Function"
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    i = 0
    collection_of_losses = []
    print(len(data_iter))
    for _, batch in enumerate(data_iter):
        #out, w2 = model.forward(batch.src, batch.trg, 
        #                    batch.src_mask, batch.trg_mask)
        #print('ITERATION: ', _)
        batch = batch.cuda()
        batch = batch.long()
        batch = batch.squeeze(0)
        PAD = 0
        trg = batch[:,:-1]
        trg_y = batch[:, 1:]
        src_mask = (batch != PAD).unsqueeze(-2)
        trg_mask = (trg != PAD).unsqueeze(-2)
        trg_mask = trg_mask & Variable(subsequent_mask(trg.size(-1)).type_as(trg_mask.data))
        ntokens = (trg_y != PAD).data.sum()
        
        out, w2 = model.forward(batch, trg, src_mask, trg_mask)
        #print("HEREEE")
        loss = loss_compute(out, trg_y, ntokens.item())

        total_loss += loss
        total_tokens += ntokens
        tokens += ntokens
        if i % 200 == 1:
            collection_of_losses.append(torch.div(loss,ntokens).cpu())
            elapsed = time.time() - start
            print("Epoch Step: %d Loss: %f Tokens per Sec: %f" %
                    (i, torch.div(loss,ntokens), tokens / elapsed))
            start = time.time()
            tokens = 0
        i += 1
    return (total_loss / total_tokens), collection_of_losses

In [5]:
# My version of Greedy Decoding
def test_my_model():

  #---------- TRAIN -----------
  #V = len(all_data.vocab) # 312
  V = 312
  criterion = LabelSmoothing(size=V, padding_idx=0, smoothing=0.0)
  model = make_model(V, V, N=2, d_model=128)
  model.cuda() # uncomment for GPU
  model_opt = NoamOpt(model.src_embed[0].d_model, 1, 400, torch.optim.Adam(model.parameters(), lr=0.6, betas=(0.9, 0.98), eps=1e-9))
  #the_train_data = data_gen(all_data.train_ds)
  the_train_data = dataloader
  collection_of_losses = np.array([])
  for epoch in range(30):
    print("epoch: ", epoch)
    model.train()
    totalLoss_div_totalTokens, collection_of_losses_inp = run_epoch(the_train_data, model, SimpleLossCompute(model.generator, criterion, model_opt))#, bad_idxs)
    #model.eval()
    #print(run_epoch(the_train_data, model, SimpleLossCompute(model.generator, criterion, None), bad_idxs))
    collection_of_losses = np.concatenate([collection_of_losses, collection_of_losses_inp])
  return model, collection_of_losses

model, losses = test_my_model()

C:\Users\Trevi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
c:\Users\Trevi\Desktop\DRAGN\MusGen\src\libs\music_transformer.py:269: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(p)


epoch:  0
1177
Epoch Step: 1 Loss: 6.319283 Tokens per Sec: 4206.986816
Epoch Step: 201 Loss: 2.556851 Tokens per Sec: 80176.312500
Epoch Step: 401 Loss: 3.002028 Tokens per Sec: 81426.406250
Epoch Step: 601 Loss: 2.258176 Tokens per Sec: 79199.179688
Epoch Step: 801 Loss: 2.394696 Tokens per Sec: 79630.898438
Epoch Step: 1001 Loss: 1.857241 Tokens per Sec: 77323.351562
epoch:  1
1177
Epoch Step: 1 Loss: 2.236180 Tokens per Sec: 78403.367188
Epoch Step: 201 Loss: 2.260035 Tokens per Sec: 77784.351562
Epoch Step: 401 Loss: 1.840687 Tokens per Sec: 76605.226562
Epoch Step: 601 Loss: 1.965867 Tokens per Sec: 79206.203125
Epoch Step: 801 Loss: 1.476752 Tokens per Sec: 79386.406250
Epoch Step: 1001 Loss: 1.594057 Tokens per Sec: 78682.539062
epoch:  2
1177
Epoch Step: 1 Loss: 1.996208 Tokens per Sec: 94860.679688
Epoch Step: 201 Loss: 0.968014 Tokens per Sec: 79606.078125
Epoch Step: 401 Loss: 1.307618 Tokens per Sec: 76144.617188
Epoch Step: 601 Loss: 1.868902 Tokens per Sec: 77760.875000


Save model to a .pt file

In [6]:
torch.save(model, '../models/new_dataset_epochs_30_LM.pt')

Load the Model from a previous state

In [ ]:
myModel = torch.load("./epochs_30_LM.pt")